<a href="https://colab.research.google.com/github/bartek-miskowiec/dw-matrix-3/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data',(X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  #MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
  ])

In [8]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train, params_fit={})

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 8s 29ms/step - loss: 2.5345 - accuracy: 0.3583 - val_loss: 0.4721 - val_accuracy: 0.8743
Epoch 2/5
272/272 [==============================] - 7s 27ms/step - loss: 0.4898 - accuracy: 0.8511 - val_loss: 0.0802 - val_accuracy: 0.9787
Epoch 3/5
272/272 [==============================] - 7s 27ms/step - loss: 0.2307 - accuracy: 0.9301 - val_loss: 0.0349 - val_accuracy: 0.9898
Epoch 4/5
272/272 [==============================] - 7s 27ms/step - loss: 0.1611 - accuracy: 0.9525 - val_loss: 0.0346 - val_accuracy: 0.9900
Epoch 5/5
272/272 [==============================] - 7s 27ms/step - loss: 0.1318 - accuracy: 0.9616 - val_loss: 0.0264 - val_accuracy: 0.9921


0.9734693877551021

In [9]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.0959 - accuracy: 0.9735


0.9734693765640259

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  #MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [15]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.43689208401008706, 'dropout_cnn_block_three': 0.439280010501527, 'dropout_cnn_block_two': 0.4709321215861476, 'dropout_dense_block_one': 0.3689508609376556, 'dropout_dense_block_two': 0.6017559808436197}
accuracy=0.945805013179779
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.42650715222620056, 'dropout_cnn_block_three': 0.46961773999654943, 'dropout_cnn_block_two': 0.43536079159889207, 'dropout_dense_block_one': 0.6703480980651533, 'dropout_dense_block_two': 0.6840507999106246}
accuracy=0.9292517304420471
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.43245385390759494, 'dropout_cnn_block_three': 0.34586945822855136, 'dropout_cnn_block_two': 0.46609728547070384, 'dropout_dense_block_one': 0.588179365657347, 'dropout_dense_block_two': 0.43900086997275356}
accuracy=0.956462562084198
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.305547489494227, 'dropout_cnn_block_three': 0.3242369819693237, 'dropout_cnn_block_two': 0.4883290567462